### Oroginal notebook https://www.kaggle.com/code/mattop/mcts-variants-initial-eda-fasteda
### Thank you wery match to Matt OP

In [2]:
!pip install fasteda

  Preparing metadata (setup.py) ... done
  Created wheel for fasteda: filename=fasteda-1.0.1-py3-none-any.whl size=4723 sha256=97d0a241fa39883c68a5a7a79554fada29c0c710e14c5dfed845813d03d8f955
  Stored in directory: /root/.cache/pip/wheels/ed/08/72/0d0ec0d3927d227b38ce9542ec3fa6b975c2e17ad0d046dcee
Successfully built fasteda


In [ ]:
!pip install h2o
!pip install -U lightautoml
!pip install transformers
!pip install -q autogluon==1.1.1

^C
ERROR: Operation cancelled by user


In [ ]:
import subprocess
import pyarrow.parquet as pq
import pandas as pd
import numpy as np
from gensim.models import FastText
from torch.utils.data import Dataset,DataLoader,SubsetRandomSampler
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
import re
import os
from os import listdir
from os.path import isfile, join
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score,recall_score,precision_score
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_percentage_error
import sklearn.metrics as metrics
from sklearn.metrics import explained_variance_score
from sklearn.metrics import max_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_poisson_deviance
from sklearn.metrics import mean_gamma_deviance
from sklearn.metrics import d2_absolute_error_score
from sklearn.metrics import d2_pinball_score
from sklearn.metrics import d2_tweedie_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import median_absolute_error,mean_absolute_error, f1_score,log_loss
from sklearn.metrics import accuracy_score,make_scorer 
from IPython.display import clear_output
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from numpy import where
import catboost
from catboost import CatBoostRegressor, Pool
import pickle  
import seaborn as sns
import tensorflow as tf
import warnings
import requests
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task
import lightautoml
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn import metrics
from autogluon.tabular import TabularDataset, TabularPredictor
import gc
import time
import h2o
from sklearn.preprocessing import LabelEncoder 
from h2o.automl import H2OAutoML
from sklearn.metrics import mean_absolute_percentage_error
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
warnings.simplefilter(action='ignore', category=FutureWarning)
device = torch.device("cuda")
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
import numpy as np
import pandas as pd
from fasteda import fast_eda
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_train = pd.read_csv("/kaggle/input/um-game-playing-strength-of-mcts-variants/train.csv", index_col = "Id")
df_test = pd.read_csv("/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv")
df_submission = pd.read_csv("/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv")
concepts = pd.read_csv("/kaggle/input/um-game-playing-strength-of-mcts-variants/concepts.csv")

In [ ]:
df_train.shape, df_test.shape, df_submission.shape

In [ ]:
#for col in ["GameRulesetName", "agent1", "agent2", "EnglishRules", "LudRules"]:
#    print(f"Column: {col} \nN unique values: {train[col].nunique()}\nExample:\n{train[col].iloc[0]}\n")
#    train = train.drop(columns = col, axis = 1)

In [ ]:
    train = df_train.copy() 
    test = df_test.copy() 
    submission  =df_submission.copy() 
    train = train[ list(train.columns)[1:]+  [list(train.columns)[0]] ]
 
    task = Task('reg', loss='mse', metric='r2') #‘binary’ 
    N_THREADS = 4
    RANDOM_STATE = 42
    TEST_SIZE = 0.2
    np.random.seed(RANDOM_STATE)
    torch.set_num_threads(N_THREADS)

  
    automl = TabularAutoML(
                task = task,
                timeout = 600*3600  ,
                cpu_limit = N_THREADS,
                general_params = {'use_algos':[['linear_l2' ,'lgb_tuned' , 'cb' , 'cb_tuned']]},#'lgb_tuned' , 'cb' , 'cb_tuned'
                selection_params ={'mode' : None},
                tuning_params = {'max_tuning_time':1200}, 
                reader_params = {'n_jobs': N_THREADS, 'cv':5, 'random_state': 42}
    )
    
    train_data=train 

   

In [ ]:
    test_predictions = automl.predict(test)
    df_test_predictions=pd.DataFrame(test_predictions.data[:,  0])

    OOF = pd.DataFrame({  'id': train_data.index.values,'utility_agent1':   out_of_fold_predictions.data[:, 0] })
    mape = mean_absolute_percentage_error( train_data['utility_agent1'].values+1, out_of_fold_predictions.data[:, 0]+1  ) 
  
    files= f'lama__{int(mape*100000)}'
    OOF.to_csv(f'{files.replace("test", "oof")}.csv', index = False)
    submission = pd.DataFrame({ 'id': test.index.values, 'utility_agent1':   df_test_predictions['utility_agent1'] })
    submission.to_csv(f'{files}.csv', index = False)  

In [ ]:
df_train

In [ ]:
    h2o.init()
    df = df_train
    label = 'utility_agent1'
    cat = [  'cap-shape', 'cap-surface', 'cap-color',
           'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color',
             'stem-root', 'stem-surface', 'stem-color',
           'veil-type', 'veil-color', 'has-ring', 'ring-type', 'spore-print-color',
           'habitat', 'season','class' ]

    hf = h2o.H2OFrame(df)
    featureColumns = train.columns
    targetColumn   = "utility_agent1"
    featureColumns.remove(targetColumn)

    train, valid = hf.split_frame(ratios = [.80], seed = 1234)
    featureColumns = train.columns
    targetColumn   = "utility_agent1"
    featureColumns.remove(targetColumn)

    aml = H2OAutoML(max_models=5, seed=1234,
                    balance_classes = False , max_runtime_secs = 81600  
                   )
    aml.train(x=featureColumns, y='utility_agent1', training_frame = train, validation_frame = valid)
    hf_submit = h2o.H2OFrame(df_test)

    for n_model in range(0,len(aml.leaderboard.as_data_frame())):
        model = h2o.get_model(aml.leaderboard.as_data_frame()['model_id'][n_model])
        name_m= list(aml.leaderboard.as_data_frame()['model_id'])[n_model][0:-len(aml.leaderboard.as_data_frame())]
        y_pred =   model.predict(hf_submit[featureColumns])  
        sol=pd.read_csv(df_submission)
        sol['utility_agent1']=  list(y_pred['predict'].as_data_frame()  ['predict'])
        sol.to_csv(f'H2O_f_{n_model}_{name_m}_.csv',index=False)

In [ ]:
fast_eda(train, correlation = False, pairplot = False)